In [ ]:
import sqlite3
print(sqlite3.sqlite_version)

3.45.3


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

# Create an engine
engine = create_engine('sqlite:///demo.db', echo=True)  # echo=True shows SQL logs

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Define a base class for models
Base = declarative_base()


In [ ]:
from sqlalchemy import Column, Integer, String, Table, ForeignKey
from sqlalchemy.orm import relationship

In [ ]:
class Product(Base):
    __tablename__ = 'products'

    p_id = Column(Integer, primary_key=True)
    name = Column(String)
    producer = Column(String)

    def __repr__(self):
        return f"<Product(p_id={self.p_id}, name='{self.name}', producer='{self.producer}')>"

In [ ]:
user_product = Table(
    'user_product', Base.metadata,
    Column('user_id', Integer, ForeignKey('users.user_id')),
    Column('product_id', Integer, ForeignKey('products.p_id'))
)


class User(Base):
    __tablename__ = 'users'
    __table_args__ = {'extend_existing': True}

    user_id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

    products = relationship('Product', secondary=user_product, backref='users')

    def __repr__(self):
        return f"<User(id={self.user_id}, name='{self.name}', age={self.age})>"

In [ ]:
Base.metadata.create_all(engine)

2025-07-25 18:10:39,849 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:10:39,849 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-07-25 18:10:39,849 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:39,855 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_product")
2025-07-25 18:10:39,855 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:39,858 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-07-25 18:10:39,858 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:39,858 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
# Create products
p1 = Product(name='iPhone', producer='Apple')
p2 = Product(name='Galaxy', producer='Samsung')

# Create user and assign products
u1 = User(name='Kuldeep', age=25, products=[p1, p2])
u2 = User(name='Vedant', age=30, products=[p2])

session.add_all([u1, u2])
session.commit()

2025-07-25 18:10:39,890 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:10:39,891 INFO sqlalchemy.engine.Engine INSERT INTO products (name, producer) VALUES (?, ?) RETURNING p_id
2025-07-25 18:10:39,899 INFO sqlalchemy.engine.Engine [generated in 0.00033s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('iPhone', 'Apple')
2025-07-25 18:10:39,903 INFO sqlalchemy.engine.Engine INSERT INTO products (name, producer) VALUES (?, ?) RETURNING p_id
2025-07-25 18:10:39,903 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Galaxy', 'Samsung')
2025-07-25 18:10:39,909 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING user_id
2025-07-25 18:10:39,909 INFO sqlalchemy.engine.Engine [generated in 0.00023s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Kuldeep', 25)
2025-07-25 18:10:39,909 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING user_id
2025-07-25 18:10:39,916 I

In [ ]:
galaxy = session.query(Product).filter_by(name='Galaxy').first()
print(galaxy.users)

2025-07-25 18:10:40,108 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:10:40,115 INFO sqlalchemy.engine.Engine SELECT products.p_id AS products_p_id, products.name AS products_name, products.producer AS products_producer 
FROM products 
WHERE products.name = ?
 LIMIT ? OFFSET ?
2025-07-25 18:10:40,115 INFO sqlalchemy.engine.Engine [generated in 0.00204s] ('Galaxy', 1, 0)
2025-07-25 18:10:40,131 INFO sqlalchemy.engine.Engine SELECT users.user_id AS users_user_id, users.name AS users_name, users.age AS users_age 
FROM users, user_product 
WHERE ? = user_product.product_id AND users.user_id = user_product.user_id
2025-07-25 18:10:40,132 INFO sqlalchemy.engine.Engine [generated in 0.00168s] (2,)
[<User(id=1, name='Kuldeep', age=22)>, <User(id=2, name='Vedant', age=21)>]


In [ ]:
for user in session.query(User).all():
    print(f"{user.name} uses:")
    for product in user.products:
        print(f"  - {product.name} by {product.producer}")

2025-07-25 18:10:40,280 INFO sqlalchemy.engine.Engine SELECT users.user_id AS users_user_id, users.name AS users_name, users.age AS users_age 
FROM users
2025-07-25 18:10:40,280 INFO sqlalchemy.engine.Engine [generated in 0.00237s] ()
Kuldeep uses:
2025-07-25 18:10:40,289 INFO sqlalchemy.engine.Engine SELECT products.p_id AS products_p_id, products.name AS products_name, products.producer AS products_producer 
FROM products, user_product 
WHERE ? = user_product.user_id AND products.p_id = user_product.product_id
2025-07-25 18:10:40,291 INFO sqlalchemy.engine.Engine [generated in 0.00179s] (1,)
  - iPhone by Apple
  - Galaxy by Samsung
Vedant uses:
2025-07-25 18:10:40,294 INFO sqlalchemy.engine.Engine SELECT products.p_id AS products_p_id, products.name AS products_name, products.producer AS products_producer 
FROM products, user_product 
WHERE ? = user_product.user_id AND products.p_id = user_product.product_id
2025-07-25 18:10:40,296 INFO sqlalchemy.engine.Engine [cached since 0.00656

In [ ]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)


2025-07-25 18:10:40,375 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:10:40,381 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-07-25 18:10:40,381 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:40,381 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_product")
2025-07-25 18:10:40,381 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:40,381 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-07-25 18:10:40,390 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:40,391 INFO sqlalchemy.engine.Engine 
DROP TABLE user_product
2025-07-25 18:10:40,391 INFO sqlalchemy.engine.Engine [no key 0.00121s] ()
2025-07-25 18:10:40,398 INFO sqlalchemy.engine.Engine 
DROP TABLE users
2025-07-25 18:10:40,398 INFO sqlalchemy.engine.Engine [no key 0.00109s] ()
2025-07-25 18:10:40,408 INFO sqlalchemy.engine.Engine 
DROP TABLE products
2025-07-25 18:10:40,408 INFO sqlalchemy.engine.Engine [no key 0.00166s] ()
2025-07-2

In [ ]:
# Recreate users
user1 = User(name='Kuldeep', age=22)
user2 = User(name='Vedant', age=21)

# Recreate products
product1 = Product(name='iPhone', producer='Apple')
product2 = Product(name='Galaxy', producer='Samsung')

# Set associations (many-to-many)
user1.products.append(product1)
user1.products.append(product2)
user2.products.append(product2)

# Add to session and commit
session.add_all([user1, user2, product1, product2])
session.commit()


2025-07-25 18:10:40,516 INFO sqlalchemy.engine.Engine INSERT INTO products (name, producer) VALUES (?, ?) RETURNING p_id
2025-07-25 18:10:40,524 INFO sqlalchemy.engine.Engine [cached since 0.624s ago (insertmanyvalues) 1/2 (ordered; batch not supported)] ('iPhone', 'Apple')
2025-07-25 18:10:40,524 INFO sqlalchemy.engine.Engine INSERT INTO products (name, producer) VALUES (?, ?) RETURNING p_id
2025-07-25 18:10:40,524 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('Galaxy', 'Samsung')
2025-07-25 18:10:40,524 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING user_id
2025-07-25 18:10:40,532 INFO sqlalchemy.engine.Engine [cached since 0.6207s ago (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Kuldeep', 22)
2025-07-25 18:10:40,532 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?) RETURNING user_id
2025-07-25 18:10:40,532 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch

C:\Users\Dell\AppData\Local\Temp\ipykernel_19244\2557265888.py:16: SAWarning: Identity map already had an identity for (<class '__main__.User'>, (1,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()
C:\Users\Dell\AppData\Local\Temp\ipykernel_19244\2557265888.py:16: SAWarning: Identity map already had an identity for (<class '__main__.Product'>, (2,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()
C:\Users\Dell\AppData\Local\Temp\ipykernel_19244\2557265888.py:16: SAWarning: Identity map already had an identity for (<class '__main__.User'>, (2,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()
C:\Users\Dell\AppData\Local\Temp\ipykernel_19244\2557265888.py:16: SAWarning: Identity map alread

In [ ]:
data = []

for user in session.query(User).all():
    for product in user.products:
        data.append({'User': user.name, 'Product': product.name, 'Producer': product.producer})

print(data)

2025-07-25 18:10:40,760 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:10:40,760 INFO sqlalchemy.engine.Engine SELECT users.user_id AS users_user_id, users.name AS users_name, users.age AS users_age 
FROM users
2025-07-25 18:10:40,760 INFO sqlalchemy.engine.Engine [cached since 0.4833s ago] ()
2025-07-25 18:10:40,760 INFO sqlalchemy.engine.Engine SELECT products.p_id AS products_p_id, products.name AS products_name, products.producer AS products_producer 
FROM products, user_product 
WHERE ? = user_product.user_id AND products.p_id = user_product.product_id
2025-07-25 18:10:40,760 INFO sqlalchemy.engine.Engine [cached since 0.4784s ago] (1,)
2025-07-25 18:10:40,760 INFO sqlalchemy.engine.Engine SELECT products.p_id AS products_p_id, products.name AS products_name, products.producer AS products_producer 
FROM products, user_product 
WHERE ? = user_product.user_id AND products.p_id = user_product.product_id
2025-07-25 18:10:40,760 INFO sqlalchemy.engine.Engine [cached since

In [ ]:
# Filter users with age > 25
users = session.query(User).filter(User.age > 15).all()
for user in users:
    print(f"{user.user_id}: {user.name} ({user.age})")

2025-07-25 18:10:41,525 INFO sqlalchemy.engine.Engine SELECT users.user_id AS users_user_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.age > ?
2025-07-25 18:10:41,525 INFO sqlalchemy.engine.Engine [generated in 0.00229s] (15,)
1: Kuldeep (22)
2: Vedant (21)


In [ ]:
# Define Order model
class Order(Base):
    __tablename__ = 'orders'
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.user_id'))
    product_id = Column(Integer, ForeignKey('products.p_id'))
    quantity = Column(Integer)

    user = relationship("User")
    product = relationship("Product")

In [ ]:
Base.metadata.create_all(engine, checkfirst=True)

2025-07-25 18:10:42,230 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:10:42,237 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2025-07-25 18:10:42,237 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:42,237 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_product")
2025-07-25 18:10:42,237 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:42,245 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-07-25 18:10:42,245 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:42,245 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-07-25 18:10:42,245 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:42,245 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2025-07-25 18:10:42,245 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:10:42,253 INFO sqlalchemy.engine.Engine 
CREATE TABLE orders (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	product_id INTEGER, 
	quantity I

In [ ]:
# Insert orders (make sure the IDs exist)
order1 = Order(user_id=1, product_id=1, quantity=2)
order2 = Order(user_id=2, product_id=2, quantity=3)

session.add_all([order1, order2])
session.commit()

2025-07-25 18:16:31,257 INFO sqlalchemy.engine.Engine INSERT INTO orders (user_id, product_id, quantity) VALUES (?, ?, ?) RETURNING id
2025-07-25 18:16:31,257 INFO sqlalchemy.engine.Engine [generated in 0.00023s (insertmanyvalues) 1/2 (ordered; batch not supported)] (1, 1, 2)
2025-07-25 18:16:31,261 INFO sqlalchemy.engine.Engine INSERT INTO orders (user_id, product_id, quantity) VALUES (?, ?, ?) RETURNING id
2025-07-25 18:16:31,261 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] (2, 2, 3)
2025-07-25 18:16:31,261 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
from sqlalchemy.orm import joinedload

results = session.query(User.name.label("User Name"),
                        Product.name.label("Product Name"),
                        Order.quantity.label("Quantity")) \
                 .join(Order, User.user_id == Order.user_id) \
                 .join(Product, Product.p_id == Order.product_id) \
                 .all()

for row in results:
    print(f"{row[0]} ordered {row[2]} units of {row[1]}")


2025-07-25 18:17:12,097 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:17:12,097 INFO sqlalchemy.engine.Engine SELECT users.name AS "User Name", products.name AS "Product Name", orders.quantity AS "Quantity" 
FROM users JOIN orders ON users.user_id = orders.user_id JOIN products ON products.p_id = orders.product_id
2025-07-25 18:17:12,097 INFO sqlalchemy.engine.Engine [cached since 248s ago] ()
Kuldeep ordered 2 units of iPhone
Vedant ordered 3 units of Galaxy


In [ ]:
from sqlalchemy import Float

class Sale(Base):
    __tablename__ = 'sales'

    id = Column(Integer, primary_key=True)
    product_id = Column(Integer, ForeignKey('products.p_id'))
    quantity = Column(Integer)
    sale_price = Column(Float)

    product = relationship("Product")


In [ ]:
Sale.__table__.create(bind=engine, checkfirst=True)


2025-07-25 18:20:32,482 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:20:32,482 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales")
2025-07-25 18:20:32,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:20:32,482 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sales")
2025-07-25 18:20:32,490 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-25 18:20:32,490 INFO sqlalchemy.engine.Engine 
CREATE TABLE sales (
	id INTEGER NOT NULL, 
	product_id INTEGER, 
	quantity INTEGER, 
	sale_price FLOAT, 
	PRIMARY KEY (id), 
	FOREIGN KEY(product_id) REFERENCES products (p_id)
)


2025-07-25 18:20:32,490 INFO sqlalchemy.engine.Engine [no key 0.00128s] ()
2025-07-25 18:20:32,490 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
sale1 = Sale(product_id=1, quantity=10, sale_price=599.99)
sale2 = Sale(product_id=1, quantity=5, sale_price=549.99)
sale3 = Sale(product_id=2, quantity=7, sale_price=999.99)

session.add_all([sale1, sale2, sale3])
session.commit()


2025-07-25 18:20:43,600 INFO sqlalchemy.engine.Engine INSERT INTO sales (product_id, quantity, sale_price) VALUES (?, ?, ?) RETURNING id
2025-07-25 18:20:43,602 INFO sqlalchemy.engine.Engine [generated in 0.00023s (insertmanyvalues) 1/3 (ordered; batch not supported)] (1, 10, 599.99)
2025-07-25 18:20:43,607 INFO sqlalchemy.engine.Engine INSERT INTO sales (product_id, quantity, sale_price) VALUES (?, ?, ?) RETURNING id
2025-07-25 18:20:43,607 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] (1, 5, 549.99)
2025-07-25 18:20:43,609 INFO sqlalchemy.engine.Engine INSERT INTO sales (product_id, quantity, sale_price) VALUES (?, ?, ?) RETURNING id
2025-07-25 18:20:43,612 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] (2, 7, 999.99)
2025-07-25 18:20:43,614 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
from sqlalchemy import func

# Total quantity sold
total_quantity = session.query(func.sum(Sale.quantity)).scalar()
print("Total Quantity Sold:", total_quantity)

# Average sale price
avg_price = session.query(func.avg(Sale.sale_price)).scalar()
print("Average Sale Price:", avg_price)

# Total revenue
total_revenue = session.query(func.sum(Sale.quantity * Sale.sale_price)).scalar()
print("Total Revenue:", total_revenue)

# Highest sale price
max_price = session.query(func.max(Sale.sale_price)).scalar()
print("Highest Sale Price:", max_price)

2025-07-25 18:21:15,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-25 18:21:15,109 INFO sqlalchemy.engine.Engine SELECT sum(sales.quantity) AS sum_1 
FROM sales
2025-07-25 18:21:15,111 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ()
Total Quantity Sold: 22
2025-07-25 18:21:15,115 INFO sqlalchemy.engine.Engine SELECT avg(sales.sale_price) AS avg_1 
FROM sales
2025-07-25 18:21:15,115 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()
Average Sale Price: 716.6566666666668
2025-07-25 18:21:15,119 INFO sqlalchemy.engine.Engine SELECT sum(sales.quantity * sales.sale_price) AS sum_1 
FROM sales
2025-07-25 18:21:15,119 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ()
Total Revenue: 15749.779999999999
2025-07-25 18:21:15,125 INFO sqlalchemy.engine.Engine SELECT max(sales.sale_price) AS max_1 
FROM sales
2025-07-25 18:21:15,127 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ()
Highest Sale Price: 999.99
